In [ ]:
from IPython.core.display import HTML
HTML('<style>.input, .jp-InputArea {display: none !important}</style>')

In [ ]:
import pandas as pd
import alpha_vantage
import csv
import requests
import warnings
import math
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from alpha_vantage.timeseries import TimeSeries
from sklearn.preprocessing import MinMaxScaler
import scipy.stats as stats
import requests
import calendar
from bs4 import BeautifulSoup
warnings.simplefilter('ignore')
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 50)

api_key = ('QFZF2CEU5FAI6YIW')
sym = input("Input your stock ticker: ")

class calls:
    
    @staticmethod
    def get_weekly_data():
        
        ts = TimeSeries(key= api_key, output_format = 'pandas')
        df, meta_data = ts.get_weekly(symbol = sym.upper())

        df.reset_index(inplace = True)
        df['date'] = pd.to_datetime(df['date'])
        df.rename(columns = {'date': 'reportedDate'}, inplace = True)
        df = df.sort_values(by = 'reportedDate')
        
        return(df)
    
    @staticmethod
    def merge_weekly_stock(frame, df, kicker): # merge stock earnings, with weekly data. Insert columns
                
        frame.sort_values(by = 'fiscalDateEnding', inplace = True)
        
        if kicker == 'TRUE':
            frame.rename(columns = {'fiscalDateEnding': 'reportedDate'}, inplace = True)
             #create Free Cash Flow 
            frame = frame.replace('None', 0)
            frame['FreeCashFlow'] = frame['operatingCashflow'] - frame['capitalExpenditures']            
        else:
            pass
            
        new_df = pd.merge_asof(frame, df, on='reportedDate', direction='nearest')

        # add in the most up to date stock price as the final column
        end_stock_date = df['reportedDate'][0]
        end_stock_val = df['4. close'][0]

        # create the dict with to be the final row with proper date, & stock close. 
        keys = list(new_df.columns)
        d = {key: None if key != '4. close' else end_stock_val for key in keys}
        d['reportedDate']= end_stock_date
        
        new_df = new_df.append(d, ignore_index = True)
        
        return(new_df)
    
    @staticmethod
    def statement(statement):
        
        url = f'https://www.alphavantage.co/query?function={statement}&symbol={sym.upper()}&apikey={api_key}'
    
        if statement == 'CASH_FLOW':
    
            r = requests.get(url)

            if r.status_code == 200:
                try:
                    cf_annual = pd.DataFrame(r.json()['annualReports'])
                except:
                    print('Annual Reports do not exist in cash flow call')
                try:
                    cf_quarterly = pd.DataFrame(r.json()['quarterlyReports'])
                except:
                    print('Quarterly Reports do not exist in cash flow call')
                    

                cf_quarterly[['paymentsForRepurchaseOfCommonStock', 'capitalExpenditures', 'operatingCashflow']] = cf_quarterly[['paymentsForRepurchaseOfCommonStock', 'capitalExpenditures', 'operatingCashflow']].replace('None', 0)
                cf_annual[['paymentsForRepurchaseOfCommonStock', 'capitalExpenditures', 'operatingCashflow']] = cf_annual[['paymentsForRepurchaseOfCommonStock', 'capitalExpenditures', 'operatingCashflow']].replace('None', 0)
                        
                return(cf_annual, cf_quarterly)
            
        elif statement == 'EARNINGS':
            
                r = requests.get(url)
                data = r.json()

                if statement == 'EARNINGS':
                    annual = pd.DataFrame(data['annualEarnings'])
                    quarterly = pd.DataFrame(data['quarterlyEarnings'])
                    annual['weekly'] = annual['fiscalDateEnding'].str[:7]
                    quarterly['weekly'] = quarterly['reportedDate'].str[:7]
                else:    
                    annual = pd.DataFrame(data['annualReports'])
                    quarterly = pd.DataFrame(data['quarterlyReports'])


                # For rows with none replace with np.nan. Then replace 'reportedEPS' with 'estimatedEPS' if NaN. 
                quarterly = quarterly.replace('None', np.nan)
                quarterly['estimatedEPS'].fillna(quarterly['reportedEPS'], inplace=True)
                annual = annual.replace('None', np.nan)     
                
                quarterly['reportedEPS'] = quarterly['reportedEPS'].astype(float)
                
                quarterly['reportedEPS_diff'] = quarterly['reportedEPS'].diff()
#                 annual['reportedEPS_diff'] = annual['reportedEPS'].diff()

                return(annual, quarterly)  #get data via alpha vantage API
        
    @staticmethod   
    def fix_dtypes(frame):
                        
        #change date_cols to datetime using regex
        date_cols = list(frame.filter(regex=re.compile('date', re.IGNORECASE)).columns)
        
#         if (frame.equals(cf_quarterly)) or (frame.equals(cf_annual)):
#             frame[['paymentsForRepurchaseOfCommonStock', 'capitalExpenditures', 'operatingCashflow']] = frame[['paymentsForRepurchaseOfCommonStock', 'capitalExpenditures', 'operatingCashflow']].replace('None', 0)
#         else:
#             pass
        
        for i in date_cols:
            frame[i] = pd.to_datetime(frame[i])
        
        #get remaining columns try to change to float, if error occurs then assume a string
        other_cols = list(set(frame.columns) - set(date_cols))

        for i in other_cols:
            try:
                frame[i] = frame[i].astype(float)
            except ValueError:
                frame[i] = frame[i].astype(str) # fix data types from the API call
    
    @staticmethod
    def scrape_splits():  # webscrape historical stock splits from said stock. 
        session = requests.Session()
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',}
        url = 'https://www.stocksplithistory.com/?symbol={}'.format(sym.upper())
        response = session.get(url, headers= headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', {'class': 'split-history-table'})
        td_tags = soup.find_all('td', {'align': 'center', 'style': 'padding: 4px; border-bottom: 1px solid #CCCCCC'})


        date_list = []
        split_list = []

        for i in range(0, len(td_tags), 2):
            date = td_tags[i].text.strip()
            split = td_tags[i+1].text.strip()
            date_list.append(date)
            split_list.append(split)

        splits = pd.DataFrame({'Date': date_list, 'Split': split_list})
        splits['Date'] = pd.to_datetime(splits['Date'])
        splits_filt = splits.loc[splits['Date'] > '01/01/2000']
        return(splits_filt)
    
    @staticmethod
    def normalize_stock_splits(df, scrape): #normalize the dataframe. 

        i = 0
        while i < len(scrape):   

            split_string = list(scrape['Split'].values)[i]
            split_list = split_string.split("for")
            split_ratio = int(split_list[0].strip())

            split_date = scrape['Date'].values[i]
            split_date = pd.to_datetime(split_date)

            #filter for current week up to split & everything prior. 
            mask = ((df['reportedDate'].dt.year < split_date.year) | 
                ((df['reportedDate'].dt.year == split_date.year) & (df['reportedDate'].dt.week <= split_date.week)))
            split_df = df.loc[mask]


            split_df['1. open'] = split_df['1. open'] / split_ratio
            split_df['2. high'] = split_df['2. high'] / split_ratio
            split_df['3. low'] = split_df['3. low'] / split_ratio
            split_df['4. close'] = split_df['4. close'] / split_ratio


            df.update(split_df)
            i += 1
        
        return(df)
            
    @staticmethod
    def plot_EPS_stock():
        fig, ax1 = plt.subplots(figsize=(20, 10))
        ax2 = ax1.twinx()

        # plot the first line with the left y-axis
        sns.lineplot(x='reportedDate', y='reportedEPS', data=df_merged, ax=ax1, color='orange', label='reportedEPS')

        # plot the second line with the left y-axis
        sns.lineplot(x='reportedDate', y='estimatedEPS', data=df_merged, ax=ax1, color='red', label='estimatedEPS')

        # plot the third with the right y-axis
        sns.lineplot(x='reportedDate', y='4. close', data=df_merged, color='green', label='Stock Price')

        ax1.set_xlabel('')
        ax1.set_ylabel('Earnings Per Share', fontsize=25)
        ax2.set_ylabel('Stock Price', fontsize=25)
        ax1.set_title('EPS & Stock Price', fontsize=25)

        # adjust the position of the legends
        legend1 = ax1.legend(loc='upper left', fontsize='x-large')
        legend2 = ax2.legend(loc='upper right', fontsize='x-large')

        # set zorder to bring the legends to the front
        legend1.set_zorder(100)
        legend2.set_zorder(100)

        last_date = df_merged['reportedDate'].iloc[-3]
        ax1.axvline(x=last_date, linestyle='dotted', color='black')

        ax1.invert_yaxis()

        # show the grid lines behind the legends
        ax1.grid(True, linestyle='dotted', zorder=0)
        ax2.grid(True, linestyle='dotted', zorder=0)

        plt.show()  # EPS vs Stock Plot #plot EPS, vs stock price weekly 
    
    @staticmethod
    def get_next_numbers():  #scraping future earnings
    
        session = requests.Session()
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',}

        url = 'https://www.zacks.com/stock/quote/{}/detailed-earning-estimates'.format(sym.upper())
        response = session.get(url, headers = headers)

        soup = BeautifulSoup(response.content, 'html.parser')

        tags = soup.find_all('a', href='/stock/chart/{}/price-consensus-eps-surprise-chart'.format(sym.upper()))

        number_values = []
        for tag in tags:
            text = tag.text
            number_values.append(text)

        t = soup.find_all('th', {'class': ''})

        tag_list = []
        for tags in t:
            tag_list.append(tags.text)

        t = pd.DataFrame(tag_list, columns = ['Tags'])
        t = pd.DataFrame(t['Tags'].drop_duplicates())
        t = t[t['Tags'].str.contains('Qtr|Year')]

        frame = {'Revenue': number_values[1:5],
                'EPS': number_values[6:10]}

        index = list(t['Tags'][0:4].values)
        scrape = pd.DataFrame(frame, index=index)
        #---------------------------------------

        tds = soup.find_all('td', {'class': 'alpha'})

        data = []       
        for td in tds:
            row = {}
            if td.text == 'Current' or td.text == '7 Days Ago' or td.text == '30 Days Ago' or td.text == '60 Days Ago' or td.text == '90 Days Ago':
                row['Period'] = td.text
                row['Value1'] = td.find_next_siblings('td')[0].text
                row['Value2'] = td.find_next_siblings('td')[1].text
                row['Value3'] = td.find_next_siblings('td')[2].text
                row['Value4'] = td.find_next_siblings('td')[3].text
                data.append(row)

        consensus = pd.DataFrame(data)
        consensus.set_index('Period', inplace = True)
        col_dict = dict(zip(list(consensus.columns), index))
        consensus = consensus.rename(columns = col_dict)

        return(scrape, consensus)
    
    @staticmethod
    def insert_proj_EPS(current_or_next, frame, proj): #insert projected EPS
    
        #get the last record of earnings quarterly and offset by 3 months
        fiscal_val = frame.iloc[len(frame) -1]['fiscalDateEnding']
        reported_val = frame.iloc[len(frame) -1]['reportedDate']
        offset = pd.DateOffset(months = 3)

        fiscal_date_end = fiscal_val+offset
        reported_date_end = reported_val + offset

        if current_or_next == 'current':
            qtr = proj['EPS'][0]
        elif current_or_next == 'next':
            qtr = proj['EPS'][1]
        else:
            print('Error with naming convention')


        first_row = pd.DataFrame({
            'fiscalDateEnding': [fiscal_date_end],
            'reportedDate': [reported_date_end],
            'reportedEPS': [None],
            'estimatedEPS': [qtr],
            'surprise': [0],
            'surprisePercentage': [0],
            'weekly': ['2023']
        })

        if current_or_next == 'current':
            frame.loc[-1] = first_row.iloc[0]
        elif current_or_next == 'next':
            frame.loc[-2] = first_row.iloc[0]
        else:
            print('Error with row update')
            
    @staticmethod
    def estimates(consensus, quarter_or_annual):  #linechart of consensus estimates EPS
    
        consensus[list(consensus.columns)] = consensus[list(consensus.columns)].astype(float)

        if quarter_or_annual == 'quarter':
            consensus = consensus.iloc[:, :2]
        elif quarter_or_annual == 'annual':
            consensus = consensus.iloc[:, 2:4]
        else:
            print('wrong input')

        sns.set(style="darkgrid")
        plt.figure(figsize=(14, 10))
        ax = sns.lineplot(data=consensus,linewidth=3.0)

        ax.invert_xaxis()

        # Set the title and labels
        plt.title(f'{quarter_or_annual.upper()} EPS Consensus Trends', fontsize = 25)
        plt.xlabel('Date', fontsize = 25)
        plt.ylabel('EPS', fontsize = 25)
        plt.legend(fontsize='x-large', loc='upper left', bbox_to_anchor=(1, 1))

        # Display the chart
        plt.show()
        
    @staticmethod
    def EPS_frame():

        df_merged.set_index('reportedDate', inplace=True)
        EPS_frame = df_merged.loc[:, ['reportedEPS', 'estimatedEPS']]

        # get the projections for next quarter to assume the empty spaces. 
        EPS_frame['reportedEPS'] = EPS_frame['reportedEPS'].fillna(EPS_frame['estimatedEPS'])
        EPS_frame = EPS_frame[['reportedEPS']]
        EPS_frame['reportedEPS'] = EPS_frame['reportedEPS'].astype(float)

        EPS_frame['percentage_change'] = EPS_frame.pct_change(periods=4)

        # Identify the indices where the reportedEPS changed from negative to positive
        positive_indices = EPS_frame.loc[(EPS_frame['reportedEPS'] > 0) & (EPS_frame['reportedEPS'].shift(4) < 0)].index
        # Update the percentage_change column to be positive for the identified indices
        EPS_frame.loc[positive_indices, 'percentage_change'] = EPS_frame.loc[positive_indices, 'percentage_change'].abs()

        EPS_frame['percentage_change'] = EPS_frame['percentage_change'] * 100
        EPS_frame['percentage_change'] = EPS_frame['percentage_change'].round(2) 
        EPS_frame_ = EPS_frame
        EPS_frame = EPS_frame[::-1]
        EPS_frame = EPS_frame.transpose()
        display(EPS_frame)
        return(EPS_frame_)
    
    @staticmethod
    def average_best_EPS():
        
        EPS_frame['quarter_groupings'] = EPS_frame.reset_index().index % 4 + 1
        grouped = EPS_frame.groupby('quarter_groupings')    

        result_df = pd.DataFrame(columns=['Month', 'EPS_mean'])  # Create an empty DataFrame

        for num in range(1, 5):

            sub = grouped.get_group(num)
            EPS_mean = round(sub['reportedEPS'].mean(), 2)
            month_name = calendar.month_name[sub.index.month.value_counts().idxmax()]

            result_df = result_df.append({'Month': month_name, 'EPS_mean': EPS_mean}, ignore_index=True)

        text = 'All Time Quarterly EPS Averages:'
        formatted_text = "\033[1;4m{}\033[0m".format(text)    
        print(f"{formatted_text}")
        
        result_df['Month_Number'] = pd.to_datetime(result_df['Month'], format='%B').dt.month
        result_df = result_df.sort_values(by = 'Month_Number')
        result_df = result_df.set_index('Month')
        display(result_df)
    
    @staticmethod
    def plot_EPS_hbar(df):

        # Set the figure size and create subplots
        fig, ax = plt.subplots(figsize=(15, 15))

        # Plot the horizontal bars with customized colors
        bars = ax.barh(df.index, df['reportedEPS'], color='black', edgecolor='black', linewidth=0.8)

        # Set the labels and title
        ax.set_xlabel('Reported EPS', fontsize=25)
        ax.set_ylabel('Reported Date', fontsize=20)
        ax.set_title('Reported EPS by Period', fontsize=25 )

        # Add data labels to the bars
        for bar in bars:
            value = bar.get_width()
            x = value
            if value < 0:
                x = 0
                ax.text(value, bar.get_y() + bar.get_height() / 2, f'{value:.2f}', va='center', ha='right', color='black')
            else:
                ax.text(value, bar.get_y() + bar.get_height() / 2, f'{value:.2f}', va='center', ha='left', color='black')

        # Customize the plot background color
        ax.set_facecolor('lightgray')

        # Customize the ticks and grid lines
        ax.tick_params(axis='both', colors='black')
        ax.grid(axis='x', linestyle='--', alpha=0.7)

        # Display the plot
        plt.tight_layout()
        plt.show()
    
# -----------------------------------------------------------------------------------------------------------------------           
    
def process_data(statement):   
    # load the initial data into the frames
    
    if statement == 'CASH_FLOW':
    
        df = calls.get_weekly_data()
        cf_annual, cf_quarterly = calls.statement(statement)

        # fix data types in place
        calls.fix_dtypes(cf_annual)
        calls.fix_dtypes(cf_quarterly)  

        # scrape and normalize stock splits
        scrape = calls.scrape_splits()
        df = calls.normalize_stock_splits(df, scrape)
        df_merge = calls.merge_weekly_stock(cf_quarterly, df, 'TRUE')
        
        return(df_merge, df)
        
    elif statement == 'EARNINGS':
        
        df = calls.get_weekly_data()
        earnings_annual, earnings_quarterly = calls.statement(statement)

        calls.fix_dtypes(earnings_annual)
        calls.fix_dtypes(earnings_quarterly)

        scrape = calls.scrape_splits()
        df = calls.normalize_stock_splits(df, scrape)
        df_merged = calls.merge_weekly_stock(earnings_quarterly, df, 'FALSE')
        
        #drop last row with close data
        df_merged = df_merged.drop(len(df_merged) - 1)
        df_merged['reportedEPS_diff'] = df_merged['reportedEPS'].diff()
        
        proj, consensus = calls.get_next_numbers()
        
        calls.insert_proj_EPS('current', df_merged, proj)
        calls.insert_proj_EPS('next', df_merged, proj)   
        
        df_merged.at[-1, '4. close'] = df.at[0, '4. close']
        
        return(df_merged, df, proj, consensus)
    
    else:
        print("issue")
    
    
df_merged, df_2, proj, consensus = process_data('EARNINGS')
df_merged_2, df_2_ = process_data('CASH_FLOW')
    
text = f"\n\n\nLast reported EPS: {df_merged.iloc[-3]['reportedEPS']} on {str(df_merged.iloc[-3]['reportedDate'])[:10]}"
formatted_text = "\033[1;4m{}\033[0m".format(text)    
print(f"{formatted_text}")

display(proj)
calls.plot_EPS_stock()
EPS_frame = calls.EPS_frame()
calls.plot_EPS_hbar(EPS_frame)
calls.average_best_EPS()

print('\n\n\n\n\n\n\n\n ')

text = 'EPS Consensus Trends'
formatted_text = "\033[1;4m{}\033[0m".format(text) 
print(f'{formatted_text}')
display(consensus)
calls.estimates(consensus, 'quarter')
calls.estimates(consensus, 'annual')

# rather than having the mean for quarterly EPS averages. DO a box plot. 
# Confirm Quarterly EPS Averages Months are correct. 


In [ ]:
def EPS_cycle():
    sub = df_merged[['fiscalDateEnding', '4. close', 'reportedEPS', 'reportedEPS_diff']]
    sub['reportedEPS'] = sub['reportedEPS'].astype(float)

    output = sub.groupby(sub['fiscalDateEnding'].dt.month)[['reportedEPS_diff']].mean()
    output.reset_index(inplace =True)
    return(output)

# ----------------------------------------------------------------------Show subplot of YoY EPS---------------------------

def plot_change(column):
    
    frame = df_merged
    frame['year'] = frame['fiscalDateEnding'].dt.year
    years = list(frame['fiscalDateEnding'].dt.year.unique())

    # Calculate the number of rows and columns needed based on the number of years
    n_rows = (len(years) - 1) // 3 + 1
    n_cols = min(len(years), 6)

    fig, axs = plt.subplots(n_rows, n_cols, figsize=(15, 2*n_rows))

    # Loop over the subplots and plot each year's data
    for i, n in enumerate(years):
        subset = frame.loc[frame['fiscalDateEnding'].dt.year == years[i]]

        row = i // n_cols
        col = i % n_cols
    
        sns.lineplot(data=subset, x='reportedDate', y=column,  ax=axs[row, col])
        #sns.scatterplot(data=subset, x='reportedDate', y=column, ax=axs[row, col], s=50, linewidth=1, edgecolor='black', zorder=3)
        axs[row, col].set_title(years[i])
        axs[row, col].set_xlabel(None)
        axs[row, col].set_ylabel(None)
        axs[row, col].set_xticklabels(axs[row, col].get_xticklabels(), rotation=45)

#     Remove any unused subplots
    for i in range(len(years), n_rows * n_cols):
        row = i // n_cols
        col = i % n_cols
        fig.delaxes(axs[row, col])

    plt.tight_layout()
    plt.show()

# -------------------------------------------------------------Correlation Average-------------------------------------

def correlation_average():

    years = list(df_merged['fiscalDateEnding'].dt.year.unique())

    for i in years:
            small = df_merged.loc[df_merged['fiscalDateEnding'].dt.year == i]
            small['reportedEPS_diff'] = small['reportedEPS_diff'].astype(float)
            try:
                out[str(i)] = small['reportedEPS_diff'].values
            except ValueError:
                pass

    corr_list = []
    year_list = []

    for year in years:
            try:

                correlation, p_value = stats.pearsonr(out['reportedEPS_diff'], out[str(year)])
                corr_list.append(correlation)
                year_list.append(year)
            except ValueError:
                pass
            except KeyError:
                pass

    corr_frame = pd.DataFrame(corr_list, index=year_list, columns=['EPS correlation YoY'])

    text = ('Earnings Cycle Correlation, takes the difference in EPS from Quarter to Quarter and compares YoY correlation')
    formatted_text = "\033[1;4m{}\033[0m".format(text)    
    print(f"{formatted_text}")

    a = [x for x in corr_list if not math.isnan(x)]

    # test correlation for most recent 60% or results
    portion = round(.60 * len(a))
    a_portion = [a[int(portion):]]
    a_portion = [item for sublist in a_portion for item in sublist]

    a = (sum(a) / len(a))
    a_portion = (sum(a_portion) / len(a_portion))

    text = (f'\nAverage correlation value for all years {round(a, 2)}')
    formatted_text = "\033[1;4m{}\033[0m".format(text)    
    print(f"{formatted_text}")

    text = (f'Average correlation value of last {str(int(portion))} years: {round(a_portion, 2)}')
    formatted_text = "\033[1;4m{}\033[0m".format(text)    
    print(f"{formatted_text}")
    
    display(corr_frame)
    
    text = ('\n** Actual EPS results typically announced 1 month after fiscal date end **')
    formatted_text = "\033[1;4m{}\033[0m".format(text)    
    print(f"{formatted_text}")

#--------------------------------------------------------------CASH FLOWS-------------------------------------------------

def cash_flows():
    df_sub = df_merged_2[['reportedDate', 'operatingCashflow', 'cashflowFromInvestment', 'cashflowFromFinancing', 'FreeCashFlow']]
    # Remove the last row with NaN values
    df_sub = df_sub[:-1]
    # Melt the dataframe to create a long-form dataframe
    df_long = pd.melt(df_sub, id_vars=['reportedDate'], var_name='Cash Flow', value_name='Amount')
    # Create the bar chart using matplotlib
    fig, ax = plt.subplots(figsize=(24, 18))
    df_long.pivot(index='reportedDate', columns='Cash Flow', values='Amount').plot(kind='bar', ax=ax)
    # Add axis labels and a title
    plt.xlabel('')
    plt.ylabel("Amount", fontsize=25)
    plt.title("Cash Flows Over Time", fontsize=25)
    ax = plt.gca()
    ax.set_xticklabels(ax.get_xticklabels(), fontsize=20)
    ax.legend(fontsize='x-large')

    ax2 = ax.twinx()
    df_merged_2['4. close'].plot(ax=ax2, color='gray', alpha=0.5, label='Stock Price')  # Include label for legend
    ax2.set_ylabel("Stock Price", fontsize=25)

    # Combine the legends from both axes
    handles1, labels1 = ax.get_legend_handles_labels()
    handles2, labels2 = ax2.get_legend_handles_labels()
    ax.legend(handles1 + handles2, labels1 + labels2, fontsize='x-large')

    plt.show()

    df_sub.set_index('reportedDate', inplace=True)
    cf_df = df_sub.loc[:, 'operatingCashflow':'FreeCashFlow']
    cf_yoy_df = cf_df.pct_change(periods=4)
    cf_yoy_df.columns = [col + '_YoY' for col in cf_yoy_df.columns]

    display(cf_yoy_df)


out = EPS_cycle()
sns.barplot(x='fiscalDateEnding', y='reportedEPS_diff', data=out)
plt.title('Average EPS Difference QoQ', fontdict={'fontsize': 14, 'fontweight': 'bold'})

correlation_average()  
plot_change('reportedEPS_diff')

cash_flows()


# Change the percentages in cash flows over time
# 2023 should still be plotted despite incomplete data. 

# Cash Flow From Invesment

Cash flow from investments refers to the amount of cash inflow or outflow generated from a company's investments in assets, such as property, equipment, securities, or other investments. Here are some examples of cash flow from investment:

Purchase of property: When a company purchases a property, the cash outflow from the investment will be recorded as a negative cash flow from investment.

Sale of property: When a company sells a property, the cash inflow from the sale will be recorded as a positive cash flow from investment.

Purchase of equipment: When a company purchases new equipment, the cash outflow from the investment will be recorded as a negative cash flow from investment.

Sale of equipment: When a company sells equipment, the cash inflow from the sale will be recorded as a positive cash flow from investment.

Purchase of securities: When a company purchases securities, such as stocks or bonds, the cash outflow from the investment will be recorded as a negative cash flow from investment.

Sale of securities: When a company sells securities, the cash inflow from the sale will be recorded as a positive cash flow from investment.

Dividend income: When a company receives dividends from its investments, the cash inflow from the dividend income will be recorded as a positive cash flow from investment.

Interest income: When a company receives interest on its investments, the cash inflow from the interest income will be recorded as a positive cash flow from investment.

# Cash Flow From Financing

Cash flow from financing refers to the inflow and outflow of cash resulting from a company's financing activities, such as borrowing money, issuing stock, or paying dividends to shareholders. These activities affect the company's capital structure and the overall amount of debt and equity that the company holds.

Here are some examples of cash flow from financing:

Issuing new stock: When a company issues new shares of stock, the cash inflow from the sale of those shares is recorded as a positive cash flow from financing.

Repurchasing stock: When a company buys back its own stock, the cash outflow from the repurchase is recorded as a negative cash flow from financing.

Issuing debt: When a company borrows money through issuing bonds or taking out loans, the cash inflow from the debt issuance is recorded as a positive cash flow from financing.

Repaying debt: When a company pays off its debt, the cash outflow from the repayment is recorded as a negative cash flow from financing.

Paying dividends: When a company pays dividends to its shareholders, the cash outflow from the payment is recorded as a negative cash flow from financing.

The net result of cash flow from financing activities can indicate whether a company is funding its operations through debt or equity, and whether it is generating enough cash to pay its obligations or requires additional financing to continue operating. Investors and analysts use this information to evaluate a company's financial health and investment potential.



